https://www.youtube.com/watch?v=l99baVWtf30&list=PLZ1QII7yudbecO6a-zAI6cuGP1LLnmW8e&index=15

In [226]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [227]:
pair = "EUR_USD"
granularity = "H1"

In [228]:

df = pd.read_csv(utils.get_hist_data_filename(pair, granularity), index_col=False)

We want most of the columns in our dataframe to be numeric so that we can do maths.  non_cols is the only 2 columns where we don't want to do this.

In [229]:
non_cols = ["time", "volume", "ticker"]
mod_cols = [x for x in df.columns if x not in non_cols]

In [230]:
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [231]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  3999 non-null   int64  
 1   ticker      3999 non-null   object 
 2   time        3999 non-null   object 
 3   volume      3999 non-null   int64  
 4   bid_o       3999 non-null   float64
 5   bid_h       3999 non-null   float64
 6   bid_l       3999 non-null   float64
 7   bid_c       3999 non-null   float64
 8   mid_o       3999 non-null   float64
 9   mid_h       3999 non-null   float64
 10  mid_l       3999 non-null   float64
 11  mid_c       3999 non-null   float64
 12  ask_o       3999 non-null   float64
 13  ask_h       3999 non-null   float64
 14  ask_l       3999 non-null   float64
 15  ask_c       3999 non-null   float64
dtypes: float64(12), int64(2), object(2)
memory usage: 500.0+ KB


In [232]:
df_plot = df.iloc[-500:].copy()

First use(s) of plotly library.  Note: "go" acronym references 'graph_objects'.  Not to be confused with Golang - which I've also been playing with recently.  It's all fun!!

In [233]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low = df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor="#24A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C"
    ))
fig.update_layout(width=1000, height=400, paper_bgcolor = "#1e1e1e", plot_bgcolor = "#1e1e1e",
                margin=dict(l=10, b=10, t=10, r=10), 
                font=dict(size=10, color="#e1e1e1"))
fig.update_xaxes(gridcolor="#1f292f",
                 showgrid=True,
                 fixedrange=True,
                 rangeslider=dict(visible=False))
fig.update_yaxes(gridcolor="#1f292f",
                 showgrid=True)
fig.show()

OK, so far so good.  Time to start building some indicators, starting with Moving Average (MA)

In [234]:
df_ma = df[["time", "ticker", "mid_o", "mid_h", "mid_l", "mid_c"]].copy()
df_ma.shape

(3999, 6)

In [235]:
df_ma["MA_8"] = df_ma.mid_c.rolling(window=8).mean()

In [236]:
df_ma.head(10)

,time,ticker,mid_o,mid_h,mid_l,mid_c,MA_8
0,2022-11-22T20:00:00.000000000Z,EUR_USD,1.02954,1.03045,1.02938,1.03006,NaN
1,2022-11-22T21:00:00.000000000Z,EUR_USD,1.03004,1.03062,1.02990,1.03026,NaN
2,2022-11-22T22:00:00.000000000Z,EUR_USD,1.03035,1.03062,1.03018,1.03041,NaN
3,2022-11-22T23:00:00.000000000Z,EUR_USD,1.03042,1.03109,1.03024,1.03108,NaN
4,2022-11-23T00:00:00.000000000Z,EUR_USD,1.03109,1.03214,1.03040,1.03055,NaN
5,2022-11-23T01:00:00.000000000Z,EUR_USD,1.03057,1.03137,1.03004,1.03111,NaN
6,2022-11-23T02:00:00.000000000Z,EUR_USD,1.03110,1.03249,1.03082,1.03240,NaN
7,2022-11-23T03:00:00.000000000Z,EUR_USD,1.03240,1.03240,1.03120,1.03157,1.030930
8,2022-11-23T04:00:00.000000000Z,EUR_USD,1.03155,1.03172,1.03102,1.03170,1.031135
9,2022-11-23T05:00:00.000000000Z,EUR_USD,1.03169,1.03312,1.03142,1.03275,1.031446


This is just me being silly:  calculate 100 (note: 99, yes I know!!) Moving Averages and store to CSV.  Probably not the most efficient way to do this - but interesting that it can be done so easily.

In [237]:
df_ma2 = df_ma.copy()
for p in range(1, 100):
    MA_Period = f"MA_{p}"
    df_ma2[MA_Period] = df_ma.mid_c.rolling(window=p).mean()

C:\Users\johnj\AppData\Local\Temp\ipykernel_2080\2932637.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [238]:
df_ma2.to_csv("./Data/Silly 100 MAs calc.csv")

Back on piste with the MA_8 work...  We don't have MA for the first 8 periods, so...

In [239]:
df_ma.dropna(inplace=True)

In [240]:
df_ma.head()

,time,ticker,mid_o,mid_h,mid_l,mid_c,MA_8
7,2022-11-23T03:00:00.000000000Z,EUR_USD,1.03240,1.03240,1.03120,1.03157,1.030930
8,2022-11-23T04:00:00.000000000Z,EUR_USD,1.03155,1.03172,1.03102,1.03170,1.031135
9,2022-11-23T05:00:00.000000000Z,EUR_USD,1.03169,1.03312,1.03142,1.03275,1.031446
10,2022-11-23T06:00:00.000000000Z,EUR_USD,1.03275,1.03362,1.03221,1.03314,1.031788
11,2022-11-23T07:00:00.000000000Z,EUR_USD,1.03312,1.03488,1.03281,1.03432,1.032193


In [241]:
df_plot = df_ma.iloc[-500:].copy()

In [242]:
fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot.MA_8, 
    line=dict(color="#027FC3", width=2),
    line_shape="spline",
    name="MA_8"))
fig.show()

In [243]:
ma_list = [8, 16, 32, 64, 128, 256]

In [244]:
for ma in ma_list:
    MA_Period = f"MA_{ma}"
    df_ma[MA_Period] = df_ma.mid_c.rolling(window=ma).mean()

In [245]:
df_ma

,time,ticker,mid_o,mid_h,mid_l,mid_c,MA_8,MA_16,MA_32,MA_64,MA_128,MA_256
7,2022-11-23T03:00:00.000000000Z,EUR_USD,1.03240,1.03240,1.03120,1.03157,NaN,NaN,NaN,NaN,NaN,NaN
8,2022-11-23T04:00:00.000000000Z,EUR_USD,1.03155,1.03172,1.03102,1.03170,NaN,NaN,NaN,NaN,NaN,NaN
9,2022-11-23T05:00:00.000000000Z,EUR_USD,1.03169,1.03312,1.03142,1.03275,NaN,NaN,NaN,NaN,NaN,NaN
10,2022-11-23T06:00:00.000000000Z,EUR_USD,1.03275,1.03362,1.03221,1.03314,NaN,NaN,NaN,NaN,NaN,NaN
11,2022-11-23T07:00:00.000000000Z,EUR_USD,1.03312,1.03488,1.03281,1.03432,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3994,2023-07-14T06:00:00.000000000Z,EUR_USD,1.12199,1.12258,1.12057,1.12102,1.122731,1.122237,1.119063,1.112165,1.104064,1.096289
3995,2023-07-14T07:00:00.000000000Z,EUR_USD,1.12102,1.12218,1.12041,1.12218,1.122711,1.122418,1.119316,1.112524,1.104324,1.096422
3996,2023-07-14T08:00:00.000000000Z,EUR_USD,1.12217,1.12361,1.12214,1.12312,1.122816,1.122577,1.119606,1.112881,1.104594,1.096563
3997,2023-07-14T09:00:00.000000000Z,EUR_USD,1.12312,1.12451,1.12263,1.12266,1.122736,1.122645,1.119861,1.113225,1.104860,1.096701


In [246]:
df_plot = df_ma.iloc[-500:].copy()
for ma in ma_list:
    ma_period = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot[ma_period], 
        line=dict(width=2),
        line_shape="spline",
        name=ma_period))

fig.show()


In [247]:
df_plot["MA_16"]

3499    1.084321
3500    1.084981
3501    1.085806
3502    1.086662
3503    1.087502
          ...   
3994    1.122237
3995    1.122418
3996    1.122577
3997    1.122645
3998    1.122632
Name: MA_16, Length: 500, dtype: float64